In [2]:
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
from functools import lru_cache

In [ ]:
df = pd.read_csv("policekillings.csv")
df.shape

In [ ]:
# filter null city values
df = df[(df["City"].notnull()) & df["State"].notnull()]
df.shape

In [ ]:
# test city conversions
address='Aurora, ' + states["CO"] + ', United States of America'
geolocator = Nominatim(user_agent="Your_Name")
location = geolocator.geocode(address, timeout = None)
print(location.address)
print((location.latitude, location.longitude))

In [ ]:
states = {'AK': 'Alaska',
            'AL': 'Alabama',
            'AR': 'Arkansas',
            'AS': 'American Samoa',
            'AZ': 'Arizona',
            'CA': 'California',
            'CO': 'Colorado',
            'CT': 'Connecticut',
            'DC': 'District of Columbia',
            'DE': 'Delaware',
            'FL': 'Florida',
            'GA': 'Georgia',
            'GU': 'Guam',
            'HI': 'Hawaii',
            'IA': 'Iowa',
            'ID': 'Idaho',
            'IL': 'Illinois',
            'IN': 'Indiana',
            'KS': 'Kansas',
            'KY': 'Kentucky',
            'LA': 'Louisiana',
            'MA': 'Massachusetts',
            'MD': 'Maryland',
            'ME': 'Maine',
            'MI': 'Michigan',
            'MN': 'Minnesota',
            'MO': 'Missouri',
            'MP': 'Northern Mariana Islands',
            'MS': 'Mississippi',
            'MT': 'Montana',
            'NA': 'National',
            'NC': 'North Carolina',
            'ND': 'North Dakota',
            'NE': 'Nebraska',
            'NH': 'New Hampshire',
            'NJ': 'New Jersey',
            'NM': 'New Mexico',
            'NV': 'Nevada',
            'NY': 'New York',
            'OH': 'Ohio',
            'OK': 'Oklahoma',
            'OR': 'Oregon',
            'PA': 'Pennsylvania',
            'PR': 'Puerto Rico',
            'RI': 'Rhode Island',
            'SC': 'South Carolina',
            'SD': 'South Dakota',
            'TN': 'Tennessee',
            'TX': 'Texas',
            'UT': 'Utah',
            'VA': 'Virginia',
            'VI': 'Virgin Islands',
            'VT': 'Vermont',
            'WA': 'Washington',
            'WI': 'Wisconsin',
            'WV': 'West Virginia',
            'WY': 'Wyoming'
    }

In [ ]:
# gen coordinates
# memoization table --> geolocator calls limited by api refresh rate
m = {}
bad_adds = []
def gen_coord(row):
    print(len(m))
    address = row["City"] + ", " + states[row["State"].upper()] + ", United States of America" 
    if address in m:
        return m[address]
    else:
        location = geolocator.geocode(address)
        if location:
            m[address] = (location.latitude, location.longitude)
        else:
            print(address)
            bad_adds.append(address)
            m[address] = None
    return m[address]

df["coordinates"] = df[["City", "State"]].apply(gen_coord, axis = 1)
df.head()

In [ ]:
print(df["coordinates"][0][0])
df = df[df["coordinates"].notna()]
df["latitude"] = df.apply(lambda row: row["coordinates"][0], axis=1)
df["longitude"] = df.apply(lambda row: row["coordinates"][1], axis=1)

In [ ]:
# now that all rows have coordinate positions convert back to csv for use
df.to_csv("police_revised.csv", index = False)

In [ ]:
tmp = pd.read_csv("police_cleaned.csv")

In [ ]:
print("unknown race count: " + str(len(tmp[tmp["Victim's race"] == "Unknown Race"].index)))
print("unknown age count: " + str(len(tmp[tmp["Victim's age"] == "Unknown"].index)))
print("non NA url: " + str(len(tmp[tmp["URL of image of victim"].notna()].index)))
print("unique names: " + str(len(tmp["Victim's name"].unique())-1))


In [ ]:
print(list(tmp.columns))

In [ ]:
# create year column 
tmp["date"] = pd.to_datetime(tmp["Date of Incident (month/day/year)"])

In [ ]:
tmp["year"] = tmp.apply(lambda row: row["date"].year, axis = 1)

In [ ]:
tmp.to_csv("police_cleaned.csv", index = False)

In [ ]:
police = pd.read_csv("police_deaths.csv")
# address='Alexandria Police department, ' + states["VA"] + ', United States of America'
# geolocator.geocode(address)
print(police.columns)
print(police.shape)
print(type(police.year[0]))
police["yearI"] = police.apply(lambda row: int(row["year"]), axis = 1)


In [ ]:
police = police[(police["state"] != "US")& (police["year"] >= 2008) ]
police.shape


In [ ]:
m = {}
bad_adds = []
def police_add_to_coords(row):
    print(len(m), len(bad_adds))
    address = row["dept"] + ", United States of America"
    if address in m:
        return m[address]
    else:
        location = geolocator.geocode(address)
        if location:
            m[address] = (location.latitude, location.longitude)
        else:
            print(address)
            bad_adds.append(address)
            m[address] = None
    return m[address]
police["coordinates"] = police.apply(police_add_to_coords, axis = 1)

In [ ]:
# address='Alexandria Police department, ' + states["VA"] + ', United States of America'
s = 'Mount Vernon Police Department, NY'
comma_i = s.rfind(",")

final = s[0:comma_i+1]+ " "+ states[s[comma_i+1:].strip()] + ", United States of America"

test = {"department": "Chippewa County Sheriff's Department"}
print(geolocator.geocode(test))

In [ ]:
police[police["coordinates"].notna()].shape
# doesn't make sense to plot by police deaths because geocoder cannot locate ~half of departments --- no cities provided only department name and state.

In [21]:
police = pd.read_csv("police_deaths.csv")
print(police.shape)
print(police.columns)

(22800, 10)
Index(['person', 'dept', 'eow', 'cause', 'cause_short', 'date', 'year',
       'canine', 'dept_name', 'state'],
      dtype='object')


In [23]:
police = police[(police["year"] >=2000)]
police.head()

,person,dept,eow,cause,cause_short,date,year,canine,dept_name,state
19943,Deputy Sheriff Ernest Martin Hull,"Greene County Sheriff's Office, NC","EOW: Sunday, January 2, 2000",Cause of Death: Automobile accident,Automobile accident,2000-01-02,2000,False,Greene County Sheriff's Office,NC
19944,Patrol Officer James Clinton Bryant,"Broxton Police Department, GA","EOW: Monday, January 3, 2000",Cause of Death: Gunfire,Gunfire,2000-01-03,2000,False,Broxton Police Department,GA
19945,"Staff Sergeant Clyde Almond ""Tub"" Merritt","Coffee County Sheriff's Office, GA","EOW: Monday, January 3, 2000",Cause of Death: Gunfire,Gunfire,2000-01-03,2000,False,Coffee County Sheriff's Office,GA
19946,Inspector Kirk B. Brookbush,"San Francisco Police Department, CA","EOW: Tuesday, January 11, 2000",Cause of Death: Aircraft accident,Aircraft accident,2000-01-11,2000,False,San Francisco Police Department,CA
19947,Police Officer James Francis Dougherty,"San Francisco Police Department, CA","EOW: Tuesday, January 11, 2000",Cause of Death: Aircraft accident,Aircraft accident,2000-01-11,2000,False,San Francisco Police Department,CA


In [24]:
police.to_csv("police_deaths_cleaned.csv")